# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda, Flatten
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division
import sys

Using TensorFlow backend.


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)

nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dict_tokens = tokenizer.word_index
dict_tokens_reverse = {v: k for k, v in dict_tokens.items()}
# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [3]:
#create co-occurrence matrix
matrix = np.zeros((V,V), dtype=float)
a = 0
maximum = 0.0
for sentence in corpus:
    for index, word in enumerate(sentence):
        j = 1
        while j <= window_size_corpus:
            if index + j < len(sentence) and sentence[index + j] != word:
                matrix[word][sentence[index + j]] = matrix[word][sentence[index + j]] + 1
                if matrix[word][sentence[index + j]] > maximum:
                    maximum = matrix[word][sentence[index + j]]
            if index - j >= 0 and sentence[index - j] != word:
                matrix[word][sentence[index - j]] = matrix[word][sentence[index - j]] + 1
                if matrix[word][sentence[index - j]] > maximum:
                    maximum = matrix[word][sentence[index - j]]
            j = j + 1

            
#normalize all words
for x in range(V):
    for y in range(V):
        matrix[x][y] = float(matrix[x][y]) / float(maximum)
        
f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V - 1),str(V)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, matrix[i])))
    f.write("\n")
f.close()

matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.93421053, ...,  0.01315789,
         0.        ,  0.        ],
       [ 0.        ,  0.93421053,  0.        , ...,  0.01315789,
         0.01315789,  0.01315789],
       ..., 
       [ 0.        ,  0.01315789,  0.01315789, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01315789, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01315789, ...,  0.        ,
         0.        ,  0.        ]])

In [4]:
#find cosine similarity to Alice, Dinah and Rabbit
alice_index = tokenizer.word_index["alice"]
dinah_index = tokenizer.word_index["dinah"]
rabbit_index = tokenizer.word_index["rabbit"]

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)
print("Alice and rabbit", co_occurrence.similarity("alice", "rabbit"))
print("Alice and dinah", co_occurrence.similarity("alice", "dinah"))
print("Rabbit and dinah", co_occurrence.similarity("rabbit", "dinah"))



Alice and rabbit 0.478909301683
Alice and dinah 0.393600110608
Rabbit and dinah 0.298623232265


In [5]:
#find the closest words to Alice
co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)
co_occurrence.similar_by_word("alice", 5)

/Users/admin/anaconda3/envs/RC/lib/python3.6/site-packages/gensim/models/keyedvectors.py:932: RuntimeWarning: invalid value encountered in true_divide
  self.vectors_norm = (self.vectors / sqrt((self.vectors ** 2).sum(-1))[..., newaxis]).astype(REAL)


[('she', 0.8023513555526733),
 ('it', 0.7543456554412842),
 ('herself', 0.7296270132064819),
 ('that', 0.725111722946167),
 ('in', 0.7187401056289673)]

Discussion of the drawbacks:

In [6]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
#f.write(" ".join([str(V - 1),str(V - 1)]))
f.write(" ".join([str(V - 1),str(V)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, matrix[i])))
    #f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [7]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [8]:
#prepare data for cbow

# EXTRA INFORMATION 
# link: http://www.claudiobellei.com/2018/01/07/backprop-word2vec-python/
# https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html
# https://github.com/nzw0301/keras-examples/blob/master/CBoW.ipynb
# http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/

def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)
            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)
            
for x,y in generate_data(corpus, window_size, V):
    if x == 'dummy loop':
        print()

/Users/admin/anaconda3/envs/RC/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [9]:
#create CBOW model
cbow1 = Sequential()
cbow1.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow1.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow1.add(Dense(V, activation='softmax'))
print(cbow1.summary())
print('check')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            118300    
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1183)              119483    
Total params: 237,783
Trainable params: 237,783
Non-trainable params: 0
_________________________________________________________________
None
check


In [10]:
#define loss function

# categorical_crossentropy is applied on the output
cbow1.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [11]:
#train model
cbow1.fit(x, y, epochs=500, verbose=1)

# print('Accuracy: %f' % (accuracy*100))
# for ite in range(1000):
#     loss = 0.
#     for x, y in generate_data(corpus, window_size, V):
#         loss += cbow.train_on_batch(x, y)
#     if ite%100 == 0:
#         print(ite, loss)

Epoch 1/500
1/1 [==============================] - 0s 125ms/step - loss: 7.0835
Epoch 2/500
1/1 [==============================] - 0s 7ms/step - loss: 7.0742
Epoch 3/500
1/1 [==============================] - 0s 8ms/step - loss: 7.0647
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 7.0551
Epoch 5/500
1/1 [==============================] - 0s 9ms/step - loss: 7.0453
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 7.0351
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 7.0244
Epoch 8/500
1/1 [==============================] - 0s 7ms/step - loss: 7.0130
Epoch 9/500
1/1 [==============================] - 0s 9ms/step - loss: 7.0009
Epoch 10/500
1/1 [==============================] - 0s 12ms/step - loss: 6.9878
Epoch 11/500
1/1 [==============================] - 0s 9ms/step - loss: 6.9735
Epoch 12/500
1/1 [==============================] - 0s 9ms/step - loss: 6.9579
Epoch 13/500
1/1 [==============================] - 0s 8ms

Epoch 105/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0110
Epoch 106/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0108
Epoch 107/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0106
Epoch 108/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0104
Epoch 109/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0102
Epoch 110/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0100
Epoch 111/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0099
Epoch 112/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0097
Epoch 113/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0095
Epoch 114/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0094
Epoch 115/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0092
Epoch 116/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0091
Epoch 117/500
1/1 [====================

Epoch 208/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0031
Epoch 209/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0031
Epoch 210/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0031
Epoch 211/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0030
Epoch 212/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0030
Epoch 213/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0030
Epoch 214/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0029
Epoch 215/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0029
Epoch 216/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0029
Epoch 217/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0028
Epoch 218/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0028
Epoch 219/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0028
Epoch 220/500
1/1 [====================

Epoch 310/500
1/1 [==============================] - 0s 7ms/step - loss: 9.1642e-04
Epoch 311/500
1/1 [==============================] - 0s 8ms/step - loss: 9.0634e-04
Epoch 312/500
1/1 [==============================] - 0s 7ms/step - loss: 8.9632e-04
Epoch 313/500
1/1 [==============================] - 0s 8ms/step - loss: 8.8540e-04
Epoch 314/500
1/1 [==============================] - 0s 5ms/step - loss: 8.7526e-04
Epoch 315/500
1/1 [==============================] - 0s 7ms/step - loss: 8.6571e-04
Epoch 316/500
1/1 [==============================] - 0s 7ms/step - loss: 8.5605e-04
Epoch 317/500
1/1 [==============================] - 0s 7ms/step - loss: 8.4603e-04
Epoch 318/500
1/1 [==============================] - 0s 7ms/step - loss: 8.3684e-04
Epoch 319/500
1/1 [==============================] - 0s 7ms/step - loss: 8.2748e-04
Epoch 320/500
1/1 [==============================] - 0s 5ms/step - loss: 8.1853e-04
Epoch 321/500
1/1 [==============================] - 0s 6ms/step - loss: 8.0

Epoch 408/500
1/1 [==============================] - 0s 8ms/step - loss: 3.6956e-04
Epoch 409/500
1/1 [==============================] - 0s 6ms/step - loss: 3.6705e-04
Epoch 410/500
1/1 [==============================] - 0s 6ms/step - loss: 3.6467e-04
Epoch 411/500
1/1 [==============================] - 0s 7ms/step - loss: 3.6210e-04
Epoch 412/500
1/1 [==============================] - 0s 7ms/step - loss: 3.5924e-04
Epoch 413/500
1/1 [==============================] - 0s 8ms/step - loss: 3.5686e-04
Epoch 414/500
1/1 [==============================] - 0s 11ms/step - loss: 3.5435e-04
Epoch 415/500
1/1 [==============================] - 0s 9ms/step - loss: 3.5209e-04
Epoch 416/500
1/1 [==============================] - 0s 10ms/step - loss: 3.4958e-04
Epoch 417/500
1/1 [==============================] - 0s 9ms/step - loss: 3.4696e-04
Epoch 418/500
1/1 [==============================] - 0s 9ms/step - loss: 3.4469e-04
Epoch 419/500
1/1 [==============================] - 0s 10ms/step - loss: 

In [13]:
#prepare data for Skipgram
def prepare_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    total = []
    all_in = []
    all_out = []
    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    in_words.append([word])
                    labels.append(sentence[i])
            if in_words != []:
                all_in.append(np.array(in_words,dtype=np.int32))
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out)
        
print('check')

check


In [15]:
#create Skipgram model
skipgram1 = Sequential()
skipgram1.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram1.add(Reshape((dim, )))
skipgram1.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
print('check')

check


In [16]:
#define loss function for Skipgram
skipgram1.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [17]:
#train Skipgram model
x,y = prepare_data_skipgram(corpus,window_size,V)
f = open('data_skipgram.txt' ,'w')

for input,outcome  in zip(x,y):
    input = np.concatenate(input)
    f.write(" ".join(map(str, list(input))))
    f.write(",")
    outcome = np.concatenate(outcome)
    f.write(" ".join(map(str,list(outcome))))
    f.write("\n")
f.close()

def generate_data_skipgram_from_file():
    f = open('data_skipgram.txt' ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram1.train_on_batch(x, y)

    print(ite, loss)

0 41248.6765487


KeyboardInterrupt: 

In [ ]:
#create CBOW model with additional dense layer
cbow2 = Sequential()
cbow2.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow2.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow2.add(Dense(V, activation='softmax'))
cbow2.add(Dense(V, activation='softmax'))
print(cbow2.summary())

In [ ]:
#define loss function for CBOW + dense
cbow2.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [ ]:
#train model for CBOW + dense
cbow2.fit(x, y, epochs=500, verbose=1)

In [ ]:
#create Skipgram with additional dense layer
skipgram2 = Sequential()
skipgram2.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram2.add(Reshape((dim, )))
skipgram2.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
skipgram2.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
print(skipgram2.summary())

In [ ]:
#define loss function for Skipgram + dense

skipgram2.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [ ]:
#train model for Skipgram + dense
skipgram2.fit(x, y, epochs=500, verbose=1)

In [ ]:
#Implement your own analogy function

def analogy_function(analogy):
    equation = analogy.split('=')[0]
    answer = analogy.split('=')[1]
    negative_eq = []
    positive_eq = []
    negative_answer = []
    positive_answer= []
    words = equation.split(' ')
    words.remove('')
    n = len(equation.split(' '))
    print(words)
    count = 0
    while count <n:
        print(n)
        if word == '-':
            negative_eq.append(words[n+1])
            count +=2
        elif word == '+':
            positive_eq.append(words[n+1])
            count += 1
        else:
            positive_eq.append(words[n])
            count += 1
    print(positive_eq)
    print(negative_eq)

analogy = 'women - men + daugher = brother'
analogy_function(analogy)

Comparison performance:

In [ ]:
# Write 


In [ ]:
#Visualization results trained word embeddings
vectors = skipgram1.get_weights()[0]
embeddings = TSNE(n_components=2).fit_transform(vectors)

def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(15,15))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

words = [dict_tokens_reverse[i] for i in range(1, V)]
plot(embeddings, words)

Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

ADVANTAGES CBOW

Designed to learn predict the correct word by a given context. 

DRAWBACK SKIPGRAM

Rare words will be ousted by more common words as results due to  infrequency

ADVANTAGES SKIPGRAM

Designed to learn to predict the correct context given a word.

DRAWBACK SKIPGRAM

Needs more data to be able to understand rare /unfreqquent words

ADVANTAGES NEGATIVE SAMPLING

The advantages of negative sampling is that not all the weights of the neurons in the network are tweaked in each sample. Only a small percentag of the weights in the network are tweaked. Speeding up the training of the neural network significantly. The other benefit of negative sampling. Negative sampling identifies words that have likely no influence on the label word and set them to 0 by identifying frequent words with the unigram distribution.


In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings